## Importing the neccessary libraries 

In [1]:
import speech_recognition as sr
import pyttsx3

### Initializing The Recognizer

In [2]:
r = sr.Recognizer()

## Now we are going to record the audio and convert this audio file into a text file

In [4]:
import pyaudio
import wave

# Parameters
FORMAT = pyaudio.paInt16  # Audio format
CHANNELS = 1  # Number of channels
RATE = 44100  # Sample rate (Hz)
CHUNK = 1024  # Chunk size (number of frames per buffer)
RECORD_SECONDS = 10  # Duration of the recording (seconds)
OUTPUT_FILENAME = "recorded_audio.wav"  # Output filename

try:
    # Initialize PyAudio
    audio = pyaudio.PyAudio()

    # Open stream
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("Recording...")

    frames = []

    # Record data
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording finished.")

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded data as a WAV file
    with wave.open(OUTPUT_FILENAME, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"Audio recorded and saved as {OUTPUT_FILENAME}")
    with sr.AudioFile(OUTPUT_FILENAME) as source:
        audio = r.record(source)  # Read the entire audio file

        try:
            # Recognize the speech using Google Web Speech API
            text = r.recognize_google(audio)
            print("Transcription: " + text)
            # Append the transcription to a text file
            with open("transcription.txt", "a") as f:
                f.write(text + "\n")
            # Save the transcription to a text file
            with open("transcription.txt", "w") as f:
                f.write(text)
                
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))
except OSError as e:
    print(f"OSError encountered: {e}")

Recording...
Recording finished.
Audio recorded and saved as recorded_audio.wav
Transcription: hello hello mike testing 123 123 123 123 123


updated for continuos recording

In [5]:
import speech_recognition as sr
import pyttsx3
import pyaudio
import wave
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from transformers import BertTokenizer, BertModel, BartForConditionalGeneration, BartTokenizer
import torch
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import threading

# Parameters
FORMAT = pyaudio.paInt16  # Audio format
CHANNELS = 1  # Number of channels
RATE = 44100  # Sample rate (Hz)
CHUNK = 1024  # Chunk size (number of frames per buffer)
RECORD_SECONDS = 10  # Duration of each recording (seconds)
OUTPUT_FILENAME = "recorded_audio.wav"  # Output filename

# Initialize the recognizer
r = sr.Recognizer()

def continuous_recording():
    """Record audio in 10-second intervals and process the transcription in real-time."""
    try:
        # Initialize PyAudio
        audio = pyaudio.PyAudio()

        # Check for input devices
        if audio.get_device_count() == 0:
            print("No input device found. Exiting.")
            return

        streams = []

        def record_audio():
            """Record audio continuously in 10-second intervals."""
            while not stop_event.is_set():
                # Open a new stream for each recording
                stream = audio.open(format=FORMAT, channels=CHANNELS,
                                    rate=RATE, input=True,
                                    frames_per_buffer=CHUNK)
                streams.append(stream)
                frames = []

                for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
                    data = stream.read(CHUNK)
                    frames.append(data)

                # Save the recorded data as a WAV file
                with wave.open(OUTPUT_FILENAME, 'wb') as wf:
                    wf.setnchannels(CHANNELS)
                    wf.setsampwidth(audio.get_sample_size(FORMAT))
                    wf.setframerate(RATE)
                    wf.writeframes(b''.join(frames))

                print(f"Audio recorded and saved as {OUTPUT_FILENAME}")

                # Stop and close the stream
                stream.stop_stream()
                streams.remove(stream)
                stream.close()

        def transcribe_audio():
            """Transcribe the recorded audio in real-time."""
            while not stop_event.is_set():
                if streams:
                    with sr.AudioFile(OUTPUT_FILENAME) as source:
                        audio_data = r.record(source)
                        try:
                            # Recognize the speech using Google Web Speech API
                            text = r.recognize_google(audio_data)
                            print("Transcription: " + text)
                            # Append the transcription to a text file
                            with open("transcription2.txt", "a") as f:
                                f.write(text + "\n")
                                # Save the transcription to a text file
                            with open("transcription2.txt", "w") as f:
                                f.write(text)
                        except sr.UnknownValueError:
                            print("Google Speech Recognition could not understand the audio")
                        except sr.RequestError as e:
                            print("Could not request results from Google Speech Recognition service; {0}".format(e))

        # Create and start the recording and transcribing threads
        record_thread = threading.Thread(target=record_audio)
        transcribe_thread = threading.Thread(target=transcribe_audio)
        record_thread.start()
        transcribe_thread.start()

        # Wait for interrupt signal to stop recording
        input("Press Enter to stop recording...\n")
        stop_event.set()

        # Ensure threads complete their work
        record_thread.join()
        transcribe_thread.join()

        # Terminate PyAudio instance
        audio.terminate()

    except OSError as e:
        print(f"OSError encountered: {e}")

# Initialize an event to handle stop signal
stop_event = threading.Event()

# Start continuous recording and transcription
continuous_recording()


: 